In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-26 04:28:29.466490: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 04:28:29.469244: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 04:28:29.500762: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 04:28:29.500807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 04:28:29.502404: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [4]:
from sklearn import preprocessing
s1 = preprocessing.MinMaxScaler()
s2 = preprocessing.MinMaxScaler()
s3 = preprocessing.MinMaxScaler()

x_train_scaled = s1.fit_transform(x_train)
x_val_scaled = s2.fit_transform(x_val)
x_test_scaled = s3.fit_transform(x_test)

x_train = np.copy(x_train_scaled)
x_val = np.copy(x_val_scaled)
x_test = np.copy(x_test_scaled)

In [5]:
model = joblib.load("/home/jovyan/WUSTL-IIoT/dl/dl_new/dnn.joblib")

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.


In [6]:
x_test_trimmed = x_test[20000:24000]
y_test_trimmed = y_test[20000:24000]

In [7]:
np.unique(y_test_trimmed)

array([0, 1, 2, 3, 4])

In [8]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=5,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [9]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.01, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_11823/1631696788.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA:   0%|          | 0/4000 [00:00<?, ?it/s]

JSMA: 100%|██████████| 4000/4000 [00:23<00:00, 167.69it/s]


In [10]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 712us/step
Accuracy: 0.99975
AUC: 1.0

macro
Precision: 0.9
Recall: 0.8
F1 Score: 0.7333333333333333

weighted
Precision: 0.999875
Recall: 0.99975
F1 Score: 0.9996666666666666

Mean FNR: 0.2
Mean TNR: 0.9999499874968741
Mean FPR: 5.001250312578145e-05
Mean TPR: 0.8


In [11]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.05, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_11823/2229238815.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:24<00:00, 164.51it/s]


In [12]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 645us/step
Accuracy: 0.9995
AUC: 0.9999499874968741

macro
Precision: 0.8666666666666666
Recall: 0.7992537313432836
F1 Score: 0.699626168224299

weighted
Precision: 0.9998333333333334
Recall: 0.9995
F1 Score: 0.9994997663551402

Mean FNR: 0.20074626865671644
Mean TNR: 0.9998999749937484
Mean FPR: 0.0001000250062515629
Mean TPR: 0.7992537313432836


In [9]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.1, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_47875/1183469131.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA:   0%|          | 0/4000 [00:00<?, ?it/s]

JSMA: 100%|██████████| 4000/4000 [00:31<00:00, 126.49it/s]


In [10]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 583us/step
Accuracy: 0.999
AUC: 0.9999499874968741

macro
Precision: 0.8400000000000001
Recall: 0.7810719131614654
F1 Score: 0.6567690253671563

weighted
Precision: 0.9997999999999999
Recall: 0.999
F1 Score: 0.9991961949265687

Mean FNR: 0.2189280868385346
Mean TNR: 0.9997999499874968
Mean FPR: 0.0002000500125031258
Mean TPR: 0.7810719131614654


In [11]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.15, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_47875/1855246553.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:31<00:00, 126.67it/s]


In [12]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 566us/step
Accuracy: 0.99925
AUC: 0.9999817176287765

macro
Precision: 0.85
Recall: 0.7818181818181819
F1 Score: 0.6704761904761904

weighted
Precision: 0.9998125
Recall: 0.99925
F1 Score: 0.9993380952380952

Mean FNR: 0.21818181818181817
Mean TNR: 0.9998499624906227
Mean FPR: 0.00015003750937734434
Mean TPR: 0.7818181818181819


In [13]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.2, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_47875/1114785325.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:31<00:00, 127.01it/s]


In [14]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 581us/step
Accuracy: 0.99775
AUC: 0.9998043638767005

macro
Precision: 0.6242481203007519
Recall: 0.7523066485753053
F1 Score: 0.6173052914625948

weighted
Precision: 0.9992793703007519
Recall: 0.99775
F1 Score: 0.9983485546912514

Mean FNR: 0.2476933514246947
Mean TNR: 0.9995463094069337
Mean FPR: 0.000453690593066273
Mean TPR: 0.7523066485753053


In [15]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.25, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_47875/2667259709.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:31<00:00, 127.34it/s]


In [16]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 561us/step
Accuracy: 0.997
AUC: 0.9993565452898526

macro
Precision: 0.6207184628237259
Recall: 0.7333785617367707
F1 Score: 0.5999149711509262

weighted
Precision: 0.9990240183792816
Recall: 0.997
F1 Score: 0.9977566504706954

Mean FNR: 0.2666214382632293
Mean TNR: 0.9993926938326222
Mean FPR: 0.0006073061673778574
Mean TPR: 0.7333785617367707


In [17]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_47875/1503088926.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:31<00:00, 127.85it/s]


In [18]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 563us/step
Accuracy: 0.9965
AUC: 0.9986266855400754

macro
Precision: 0.6207070707070708
Recall: 0.7318860244233378
F1 Score: 0.5991546433651698

weighted
Precision: 0.999020202020202
Recall: 0.9965
F1 Score: 0.997501940662467

Mean FNR: 0.2681139755766621
Mean TNR: 0.9992926688263706
Mean FPR: 0.0007073311736294203
Mean TPR: 0.7318860244233378
